# Artificial Neural Network #
Full implementation of artificial neurons and MLPs without the use of in built functions or related libraries


## Library Imports ##
* Numpy
* Pandas
* Mathplot

In [ ]:
import numpy as np                         # For Array Operations
import pandas as pd                        # For DataFrame Operations
import matplotlib.pyplot as plt            # For Plotting

### Defining Global Functions ###
| Function Name          | Use                                |
| :--------------------: | :--------------------------------: |
|activation_function     | Defines few activation functions   |
|activation_derivative   | Pre-evaluated derivatives          |
|loss_function           | Defines few loss functions         |
|loss_derivative         | Pre-evaluated derivatives          |


**Available activation functions :**
* Sigmoid
* Tanh
* ReLU
* Leaky ReLU
* Softmax


**Available Loss functions :**
* MSE
* Binary Cross Entropy
* Categorical Cross Entropy


In [ ]:
# Defining the activation functions and their derivatives
def activation_function(function_name):
    try:                                                                  
        if (function_name.lower() == 'sigmoid'):
            return lambda x: 1 / (1 + np.exp(-x))
        elif (function_name.lower() == 'tanh'):
            return lambda x: np.tanh(x)
        elif (function_name.lower() == 'relu'):
            return lambda x: np.maximum(0, x)   
        elif (function_name.lower() == 'leaky_relu'):
            return lambda x: np.where(x > 0, x, 0.01 * x)
        elif (function_name.lower() == 'softmax'):
            return lambda x: np.exp(x) / np.sum(np.exp(x), axis=0)
    except Exception as e:
        print(f"Error finding activation function: {e}")
        return None
        
def activation_derivative(function_name):
    try:
        if (function_name.lower() == 'sigmoid'):
            return lambda x: x * (1 - x)
        elif (function_name.lower() == 'tanh'):
            return lambda x: 1 - np.tanh(x) ** 2
        elif (function_name.lower() == 'relu'):
            return lambda x: np.where(x > 0, 1, 0)
        elif (function_name.lower() == 'leaky_relu'):
            return lambda x: np.where(x > 0, 1, 0.01)
        elif (function_name.lower() == 'softmax'):
            return lambda x: x * (1 - x)
    except Exception as e:
        print(f"Error finding activation derivative function: {e}")
        return None
        
# Defining the loss functions and their derivatives      
def loss_function(function_name):
    try:
        if (function_name.lower() == 'mean_squared_error'):
            return lambda y_true, y_pred: np.mean((y_true - y_pred) ** 2)
        elif (function_name.lower() == 'binary_crossentropy'):
            return lambda y_true, y_pred: -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        elif (function_name.lower() == 'categorical_crossentropy'):
            return lambda y_true, y_pred: -np.sum(y_true * np.log(y_pred), axis=1).mean()
    except Exception as e:
        print(f"Error finding loss function: {e}")
        return None

def loss_derivative(function_name):
    try:
        if (function_name.lower() == 'mean_squared_error'):
            return lambda y_true, y_pred: 2 * (y_pred - y_true) / y_true.size
        elif (function_name.lower() == 'binary_crossentropy'):
            return lambda y_true, y_pred: -(y_true / y_pred) + ((1 - y_true) / (1 - y_pred))
        elif (function_name.lower() == 'categorical_crossentropy'):
            return lambda y_true, y_pred: -y_true / y_pred
        elif (function_name.lower() == 'crossentropy'):
            return lambda y_true, y_pred: -y_true / y_pred
    except Exception as e:
        print(f"Error finding loss derivative function: {e}")
        return None

### Layer Class ###
To utilise a deep network of neuron layers, we can define a layer class with variable parameters like, number of neurons in the layer (inputs) and the number of outputs

In [ ]:
class Layer:
    # Initializing the layer with weights and biases with a default activation function (Sigmoid)
    def __init__(self, input_size, output_size, function_name='sigmoid'):
        self.W = np.random.randn(output_size, input_size) * 0.01    # Initializing weights with small random values
        self.B = np.zeros((output_size, 1))                         # Initializing biases with zeros
        self.function_name = function_name                          # Storing the activation function name
        
    # Defining the feed forward function for the layer
    def feed_forward(self, X):
        self.Z = np.dot(self.W, X) + self.B                         # Simply implementing the formula Z = WX + B
        self.A = activation_function(self.function_name)(self.Z)    # Applying the activation function to Z to get A
        return self.A
    
    # Defining the back propagation function for the layer
    def back_propagation(self, dA, X):
        m = X.shape[1]                                              # To utilise batch processing
        dL = dA * activation_derivative(self.function_name)(self.A) # Using the chain rule to calculate the derivative of the loss with respect to Z - (dL = dL/dZ)
        self.dW = np.dot(dL, X.T) / m                               # Gradient of the loss with respect to W - (dL/dW = dL/dZ * dZ/dW)
        self.dB = np.sum(dL, axis=1, keepdims=True) / m             # Gradient of the loss with respect to B - (dL/dB = dL/dZ * dZ/dB)
        dX = np.dot(self.W.T, dL)                                   # Gradient of the loss with respect to Input - (dL/dX = dL/dZ * dZ/dX)          
        
        return dX                                                   # Return the gradient of the loss with respect to the input for the next layer

In [13]:
Layer1 = Layer(3,2)
input_data = np.array([[1], [4], [7]])
true_output = np.array([[0], [1]])


print()
print("Input Data:")
print(input_data)
print()
print("Required Output")
print(true_output)
print()
print("Forward Output")
L1 = Layer1.feed_forward(input_data)
loss_value = loss_function('mean_squared_error')(true_output, L1)
dA = loss_derivative('mean_squared_error')(true_output, L1)

print(L1)
print()

print(f"Loss Value: {loss_value :.4f}")
print()
print("Back Output")
print(Layer1.back_propagation(dA, input_data))
print()


Layer1.W -= Layer1.dW
Layer1.B -= Layer1.dB
print("Forward Output")
L1 = Layer1.feed_forward(input_data)
loss_value = loss_function('mean_squared_error')(true_output, L1)
dA = loss_derivative('mean_squared_error')(true_output, L1)

print(L1)

print()

print(f"Loss Value: {loss_value :.4f}")
print()
print("Back Output")
print(Layer1.back_propagation(dA, input_data))
print()


test_data = np.array([[2], [8], [14]])
L2 = Layer1.feed_forward(test_data)
print("Test Data")
print(L2)


Weights:
[[-0.00265736 -0.02004092  0.016813  ]
 [ 0.01453095 -0.00515618 -0.00237033]]
Biases:
[[0.]
 [0.]]

Input Data:
[[1]
 [4]
 [7]]

Required Output
[[0]
 [1]]

Forward Output
[[0.50871661]
 [0.49432873]]

Loss Value: 0.2572

Back Output
[[-0.00217459]
 [-0.00189626]
 [ 0.00243723]]

Forward Output
[[2.06801652e-04]
 [9.99785335e-01]]

Loss Value: 0.0000

Back Output
[[-1.20428216e-08]
 [-4.56583254e-08]
 [-7.79900935e-08]]

Test Data
[[4.85852061e-08]
 [9.99999948e-01]]
